📘 IPC Section Assistant (Bot 1)

This mode helps you understand sections of the Indian Penal Code (IPC) in a clear, structured format.

✅ What it provides:

Section number

Description of the section

Nature of the offense

Punishment under the law

🧠 Example Questions:

"Explain IPC 302"

"What is the punishment under IPC 375?"

"Details of IPC 420"

⚖️ Ideal for:

Students, legal aspirants, or anyone curious about Indian criminal laws.



🔐 Requirements Before You Start

To use this chatbot, you’ll need to create free accounts and obtain API keys from the following services:

1️⃣ Cohere API Key

Used for understanding and answering questions about IPC sections.

📍 Sign up at: https://dashboard.cohere.com

🔑 Get your API key from the API Keys section after logging in

2️⃣ Serper API Key

Used for retrieving real-time news about crime cases from the web.

📍 Sign up at: https://serper.dev

🔑 Go to your Dashboard to copy your API key

💡 How to Use the Keys:

When prompted by the chatbot, paste your API key for each service.

Your keys remain local in your program and are not shared externally.

In [ ]:

!pip install faiss-cpu
# Make sure to install necessary packages
!pip install langchain cohere kagglehub pandas

In [ ]:


import os
import kagglehub
import pandas as pd
from langchain_community.llms import Cohere
from langchain.vectorstores import FAISS
from langchain.embeddings import CohereEmbeddings
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA

#for creating prompt template
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Step 1: Get Cohere API key from user
cohere_api_key = input("🔑 Enter your Cohere API key: ").strip()

# Step 2: Download dataset from KaggleHub
dataset_path = kagglehub.dataset_download("dev523/indian-penal-code-ipc-sections-information")
print("Downloaded dataset at:", dataset_path)

# Step 3: Read dataset using pandas
csv_file = os.path.join(dataset_path, "ipc_sections.csv")  # Replace with actual file if different
df = pd.read_csv(csv_file)
print(df.columns)
print(df.head())


🔑 Enter your Cohere API key: tu0ftMJVAifNZHNUesdRRudZr7yX5zD6mtN06LpR
Downloaded dataset at: /kaggle/input/indian-penal-code-ipc-sections-information
Index(['Description', 'Offense', 'Punishment', 'Section'], dtype='object')
                                         Description  \
0  Description of IPC Section 140\nAccording to s...   
1  Description of IPC Section 127\nAccording to s...   
2  Description of IPC Section 128\nAccording to s...   
3  Description of IPC Section 129\nAccording to s...   
4  Description of IPC Section 130\nAccording to s...   

                                             Offense  \
0  Wearing the dress or carrying any token used b...   
1  Receiving property taken by war or depredation...   
2  Public servant voluntarily allowing prisoner o...   
3  Public servant negligently suffering prisoner ...   
4  Aiding escape of, rescuing or harbouring, such...   

                                 Punishment  Section  
0                  3 Months or Fine or Both  I

In [ ]:

# Combine all rows into one text block (or create multiple documents if preferred)
documents = []


for _, row in df.iterrows():
    section_text = (
        f"Section {str(row.get('Section', '')).strip()}\n\n"
        f"Description:\n{str(row.get('Description', '')).strip()}\n\n"
        f"Offense:\n{str(row.get('Offense', '')).strip()}\n\n"
        f"Punishment:\n{str(row.get('Punishment', '')).strip()}"
    )
    documents.append(Document(page_content=section_text))



In [ ]:



# Step 4: Initialize Cohere embeddings
embeddings = CohereEmbeddings(
    cohere_api_key=cohere_api_key,
    user_agent="ipc_qa_bot/1.0"
)

# Step 5: Create vector store
vectorstore = FAISS.from_documents(documents, embeddings)

# Step 6: Initialize Cohere LLM
llm = Cohere(cohere_api_key=cohere_api_key, model="command")


prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a legal assistant specializing in the Indian Penal Code (IPC). Use the following structured data context to answer the user's question in a **clear and structured format**.

Each IPC section includes in separate line:
- Section: IPC section number
- Description: Explanation of the section
- Offense: Nature of the offense
- Punishment: Penalties associated with the offense

Respond by clearly organizing the answer under the following headings:

Section:
Description:
Offense:
Punishment:

Context:
{context}

Question:
{question}

Structured Answer:
"""
)


# Step 8: Build RetrievalQA with custom prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_template}
)

# Step 9: Chat loop
print("\nAsk questions about the Indian Penal Code (type 'exit' to quit):")
while True:
    query = input("Question:(type exit if you want to exit) ")
    if query.lower() == "exit":
        print("Goodbye!")
        break
    response = qa_chain.run(query)
    print(f"Answer: {response}\n")



Ask questions about the Indian Penal Code (type 'exit' to quit):
Question:(type exit if you want to exit) section 234
Answer:  Section: IPC 234
Description: Manufacturing, dealing with instruments for counterfeiting Indian coins
Offense: Making, mending, buying, selling Instruments for the purpose of counterfeiting Indian coin
Punishment: 7 Years + Fine

Question:(type exit if you want to exit) rape
Answer:  
```
Section: IPC_375
Description: Committing rape, which involves non-consensual sexual activity including penetration, object insertion, manipulation of body parts, or using mouth, against a woman's will, without her consent, or in cases where she is unable to communicate consent or is under 18 years of age.
Offense: Rape
Punishment: Penalties associated with the offense include rigorous imprisonment for a term which shall not be less than twenty years, but which may extend to life which shall mean imprisonment for the remainder of that person's natural life, and with fine;
Sect

📰 Case News Assistant (Bot 2)

This model helps you get summarized, up-to-date information about real crime cases that are currently being discussed in India or globally.

🔍 What it can do:

Find recent news articles about real-world criminal cases (e.g., rape, murder, scams).


Provide a brief summary of the case based on available news.

Show 2–3 recent developments or headlines in bullet points.

Include clickable links to trusted sources.

✅ Example Questions:

"Delhi minor rape case"

"What’s the latest in Shraddha Walkar murder case?"

"News about Manipur violence"

"Bangalore child kidnapping incident"

"Arrest in Kanjhawala case"

🧠 How it works:

Uses the Serper API (powered by Google News) to search for real-time news.

A language model (LLM) extracts:

What the case is about

Latest updates (in bullet points with links)

⚠️ Limitations:

Relies on publicly available news—older or less-known cases might return limited data.

Doesn’t access government crime databases or court judgments.

In [ ]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool, initialize_agent, AgentType

# Prompt user for SERPER key
serper_api_key = input("🔑 Enter your Serper API key: ").strip()

# Initialize wrapper with explicit key
news_wrapper = GoogleSerperAPIWrapper(serper_api_key=serper_api_key)

# Define the Tool using the wrapper
news_tool = Tool(
    name="News Search",
    func=news_wrapper.run,
    description="Searches for real-time news articles about specific crime cases using Google News"
)

# Initialize agent with tool + LLM
agent = initialize_agent(
    tools=[news_tool],
    llm=llm,  # already initialized Cohere model
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    handle_parsing_errors=True
)


🔑 Enter your Serper API key: c2f65ea4b9e2b93ff34d4c4dd647535ad6e2a149


In [ ]:
print("\nAsk about specific crime cases to get recent news (type 'exit' to quit):")
while True:
    query = input("Question: ")
    if query.lower() == "exit":
        print("Goodbye!")
        break

    print("🔎 Searching for recent news related to the case...")
    try:
        # chnged this template as it was taking to much credit
        # response = agent.run(f"""
        # Summarize what the case "{query}" is about in 1–2 lines, followed by  recent news updates related to it.
        # Present the news in bullet points. Each point should include a short summary and a clickable link to the source.
        # """
        # )

        response = agent.run(f"""
        Summarize what the case "{query}" is about in 1–2 lines
        """
        )

    except Exception as e:
        response = f"⚠️ Error fetching news: {e}"

    print(f"\n🧠 Answer:\n{response}\n")



Ask about specific crime cases to get recent news (type 'exit' to quit):
Question: Shraddha Murder Case
🔎 Searching for recent news related to the case...

🧠 Answer:
The "Shraddha Murder Case" is about a mysterious murder of a woman, Shraddha, in Delhi that involves a man, Aftab, who allegedly committed the murder and is still under investigation for the motive and reason behind it.

Question: exit
Goodbye!


In [ ]:
print("""
Welcome! 👋

What would you like to explore today?

1️⃣ IPC Sections — Learn about Indian Penal Code laws (e.g., IPC 302, IPC 375)
2️⃣ Real-World Crime Cases — Get recent news and updates on crime cases (e.g., Delhi minor rape case)

""")

mode = input("Type 'ipc' for IPC sections or 'case' for real-world cases: ").strip().lower()

if mode == "ipc":
    print("\nYou selected IPC mode. Ask about IPC sections (e.g., 'Explain IPC 375'):")
    while True:
        query = input("Question: ")
        if query.lower() == "exit":
            print("Goodbye!")
            break
        try:
            response = qa_chain.run(query)
        except Exception as e:
            response = f"⚠️ Error: {e}"
        print(f"\n🧠 Answer:\n{response}\n")

elif mode == "case":
    print("\nYou selected Case News mode. Ask about real-world crime cases (e.g., 'Delhi rape case'):")
    while True:
        query = input("Question: ")
        if query.lower() == "exit":
            print("Goodbye!")
            break
        try:
            print("🔎 Searching for recent news...")
            response = agent.run(
                f"""Summarize what the case "{query}" is about in 1–2 lines, followed by 2–3 recent news updates.
Return the news in bullet points with clickable links."""
            )
        except Exception as e:
            response = f"⚠️ Error fetching news: {e}"
        print(f"\n🧠 Answer:\n{response}\n")

else:
    print("❌ Invalid choice. Please restart the program and type 'ipc' or 'case'.")
